Importing packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Reading in the data

In [2]:
df = pd.read_csv('../../../data/WA_Fn-UseC_-Telco-Customer-Churn.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


Making charges numeric and mapping senior citizen to be consistient with other categorical columns.

In [3]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

In [4]:
df['SeniorCitizen'] = df['SeniorCitizen'].map(lambda x: 'No' if x == 0 else 'Yes')

Lowercase column names so that they are easier to work with.

In [5]:
df.columns = df.columns.str.lower()

Lets see how many unique values there are in each column.

In [6]:
for col in df.columns:
    print(col,df[col].nunique())

customerid 7043
gender 2
seniorcitizen 2
partner 2
dependents 2
tenure 73
phoneservice 2
multiplelines 3
internetservice 3
onlinesecurity 3
onlinebackup 3
deviceprotection 3
techsupport 3
streamingtv 3
streamingmovies 3
contract 3
paperlessbilling 2
paymentmethod 4
monthlycharges 1585
totalcharges 6530
churn 2


Most columns are categorical so it makes sense that those don't have many uniquw values. Customer id's are all unique and the rest are numerical columns. First, pull out the customer id's so we can match them up with model predictions later. Then, make some engineered features.

In [7]:
cust_id = df['customerid']
df = df.drop('customerid',axis=1)

df['totchg_per_tenure'] = df['totalcharges'] / df['tenure']
df['monthly_div_tot'] = df['monthlycharges'] / df['totalcharges']


Griffin's engineered features

In [8]:
df['partner_cont'] = df['partner'].map({'No': 0, 'Yes': 1, })
df['dependent_cont'] = df['dependents'].map({'No': 0, 'Yes': 1, })
df['single_parent'] = df['partner_cont'] + df['dependent_cont']
df['single_parent'] = df['single_parent'].map({1: 'No', 0: 'No', 2: 'Yes' })
df.drop(['partner_cont', 'dependent_cont'], axis=1, inplace=True)
df['phone_count'] = df['phoneservice'].map({'No': 0, 'Yes': 1, })
df['internet_count'] = df['internetservice'].map({'No': 0, 'DSL': 1, 'Fiber optic': 1 })
df['phone_and_internet'] = df['phone_count'] + df['internet_count']
df['phone_and_internet'] = df['phone_and_internet'].map({1: 'No', 0: 'No', 2: 'Yes' })
df.drop(['phone_count', 'internet_count'], axis=1, inplace=True)

In [9]:
df['o_sec'] = df['onlinesecurity'].map({'No': 0, 'Yes': 1})
df['o_back'] = df['onlinebackup'].map({'No': 0, 'Yes': 1})
df['t_sup'] = df['techsupport'].map({'No': 0, 'Yes': 1})
df['tv'] = df['streamingtv'].map({'No': 0, 'Yes': 1})
df['movie'] = df['streamingmovies'].map({'No': 0, 'Yes': 1})
df['protect'] = df['deviceprotection'].map({'No': 0, 'Yes': 1})
df['num_of_internet_services'] = df['o_sec'] + df['o_back'] + df['t_sup'] + df['tv'] + df['movie']
df['num_of_services'] = df['num_of_internet_services'] + df['protect']
df.drop(['o_back', 't_sup', 'tv', 'movie','protect'], axis=1, inplace=True)
df.drop('o_sec', axis=1, inplace=True)

In [10]:
df['num_of_ser_times_tot_charges'] = df['num_of_services'] * df['totalcharges']

In [11]:
df['tenure_times_tot_charges'] = df['tenure'] * df['totalcharges']

Make lists that have each type of column. This organizes the columns for imputing, scaling, and encoding.

In [12]:
ord_encode = ['gender','partner','dependents','phoneservice','multiplelines','internetservice','onlinesecurity','onlinebackup',
              'deviceprotection','techsupport','streamingtv','streamingmovies','contract','paperlessbilling','paymentmethod','single_parent','phone_and_internet','seniorcitizen']
labelencode =['churn']
num = ['tenure','monthlycharges','totalcharges','totchg_per_tenure','monthly_div_tot','num_of_internet_services','num_of_services','num_of_ser_times_tot_charges']

Make feature and target sets and then make train, validation, and test sets.

In [13]:
X = df[ord_encode].join(df[num],how='left')
X = X.join(cust_id,how='left')
y = df[labelencode]

from sklearn.model_selection import train_test_split

X_t, X_test, y_t, y_test = train_test_split(X,y,random_state=0)

In [14]:
X_train, X_val, y_train, y_val = train_test_split(X_t,y_t,random_state=0)

Impute missing values as 0

In [15]:
from sklearn.impute import SimpleImputer

impute = SimpleImputer(strategy='constant',fill_value=0)
X_train_im = impute.fit_transform(X_train)
X_val_im = impute.transform(X_val)

X_test_im = impute.transform(X_test)

X_train_im = pd.DataFrame(X_train_im,columns=X.columns)
X_val_im = pd.DataFrame(X_val_im,columns=X.columns)

X_test_im = pd.DataFrame(X_test_im,columns=X.columns)

In [16]:
X_train_im.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3961 entries, 0 to 3960
Data columns (total 27 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   gender                        3961 non-null   object
 1   partner                       3961 non-null   object
 2   dependents                    3961 non-null   object
 3   phoneservice                  3961 non-null   object
 4   multiplelines                 3961 non-null   object
 5   internetservice               3961 non-null   object
 6   onlinesecurity                3961 non-null   object
 7   onlinebackup                  3961 non-null   object
 8   deviceprotection              3961 non-null   object
 9   techsupport                   3961 non-null   object
 10  streamingtv                   3961 non-null   object
 11  streamingmovies               3961 non-null   object
 12  contract                      3961 non-null   object
 13  paperlessbilling  

In [17]:
from sklearn.preprocessing import OrdinalEncoder
#ord encode categoticals
ordi = OrdinalEncoder(dtype=np.int)

X_train_ord = ordi.fit_transform(X_train_im[ord_encode])
X_val_ord = ordi.transform(X_val_im[ord_encode])

X_test_ord = ordi.transform(X_test_im[ord_encode])

X_train_ord = pd.DataFrame(X_train_ord,columns=X[ord_encode].columns)
X_val_ord = pd.DataFrame(X_val_ord,columns=X[ord_encode].columns)

X_test_ord = pd.DataFrame(X_test_ord,columns=X[ord_encode].columns)

from sklearn.preprocessing import StandardScaler
#scale numerical
#use median for limit effect of outliers
ss = StandardScaler()

X_train_num = ss.fit_transform(X_train_im[num])
X_val_num = ss.transform(X_val_im[num])

X_test_num = ss.transform(X_test_im[num])

X_train_num = pd.DataFrame(X_train_num,columns=X[num].columns)
X_val_num = pd.DataFrame(X_val_num,columns=X[num].columns)

X_test_num = pd.DataFrame(X_test_num,columns=X[num].columns)

#add customer id
X_train_num = X_train_num.join(X_train_im['customerid'],how='right')
X_val_num = X_val_num.join(X_val_im['customerid'],how='right')

X_test_num = X_test_num.join(X_test_im['customerid'],how='right')

In [18]:
X_val_num

,tenure,monthlycharges,totalcharges,totchg_per_tenure,monthly_div_tot,num_of_internet_services,num_of_services,num_of_ser_times_tot_charges,customerid
0,0.022881,-1.489476,-0.712634,-1.471300,-0.449101,-1.095781,-1.107179,-0.683896,3769-MHZNV
1,-0.344542,0.547812,-0.157005,0.518759,-0.405672,1.504422,1.077941,0.002328,4277-UDIEF
2,-1.120212,-0.009181,-0.853459,0.198523,0.097378,0.854371,0.531661,-0.589555,7905-TVXTA
3,-1.283510,0.313641,-0.976974,0.313909,3.027120,-0.445730,-0.560899,-0.677297,8687-BAFGU
4,1.370097,-0.071069,0.823734,-0.062752,-0.502407,2.154473,2.170501,1.529241,8404-VIOMB
...,...,...,...,...,...,...,...,...,...
1316,-1.161036,-1.504530,-0.979833,-1.585227,0.485920,-1.095781,-1.107179,-0.683896,0690-SRQID
1317,0.349479,-1.516238,-0.667400,-1.520334,-0.467010,-1.095781,-1.107179,-0.683896,5465-BUBFA
1318,-0.875263,0.721767,-0.613853,0.553328,-0.210706,-0.445730,-0.560899,-0.604246,3797-FKOGQ
1319,1.615045,0.766929,1.855095,0.838168,-0.508011,2.154473,2.170501,2.774144,4077-HWUYD


In [19]:
#combine the categorical and numerical dfs

X_train_sc = X_train_ord.join(X_train_num,how='left')
X_val_sc = X_val_ord.join(X_val_num,how='left')

X_test_sc = X_test_ord.join(X_test_num,how='left')

In [20]:
#encode the labels

from sklearn.preprocessing import LabelEncoder

lbl = LabelEncoder()

y_train = lbl.fit_transform(np.array(y_train))
y_val = lbl.transform(y_val)

y_test = lbl.transform(y_test)

y_train_enc = pd.DataFrame(y_train,columns=y.columns)
y_val_enc = pd.DataFrame(y_val,columns=y.columns)

y_test_enc = pd.DataFrame(y_test,columns=y.columns)

C:\Users\gnr40\anaconda3\envs\learn-env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [21]:
y_test_enc.value_counts()

churn
0        1298
1         463
dtype: int64

In [22]:
X_train_sc.to_csv('new_X_train_sc.csv',index=False)
X_val_sc.to_csv('new_X_val_sc.csv',index=False)

y_train_enc.to_csv('new_y_train_enc.csv',index=False)
y_val_enc.to_csv('new_y_val_enc.csv',index=False)

X_test_sc.to_csv('new_X_test_sc.csv',index=False)
y_test_enc.to_csv('new_y_test_enc.csv',index=False)

In [23]:
X_val_sc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1321 entries, 0 to 1320
Data columns (total 27 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   gender                        1321 non-null   int32  
 1   partner                       1321 non-null   int32  
 2   dependents                    1321 non-null   int32  
 3   phoneservice                  1321 non-null   int32  
 4   multiplelines                 1321 non-null   int32  
 5   internetservice               1321 non-null   int32  
 6   onlinesecurity                1321 non-null   int32  
 7   onlinebackup                  1321 non-null   int32  
 8   deviceprotection              1321 non-null   int32  
 9   techsupport                   1321 non-null   int32  
 10  streamingtv                   1321 non-null   int32  
 11  streamingmovies               1321 non-null   int32  
 12  contract                      1321 non-null   int32  
 13  pap

Used this notebook to add my experimental features to the split data